In [ ]:
import logging
format_log = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(filename='build_cross_validate_reports.log', format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.DEBUG, datefmt='%m/%d/%Y %I:%M:%S %p')

# Build Reports for Dialogflow K-fold Cross Validation

### Overview
This is Jupyter Notebook to build reports to analyze the results of Dialogflow k-fold cross validation in the web app and Excel

### Notebook Steps
1. Loads the current and previous k-fold cross validation data
2. Builds dataframes that will be used to output the reports
3. Builds and combines the previous k-fold reports for comparision
4. Allows flags and thresholds to be added to the report.
5. Creates a confusion matrix of prediction
6. Outputs the data to Excel for analysis and the web app to analyze intents




In [ ]:
# load libraries
logging.info("building reports for k-fold cross-validation started")

import pandas as pd
import numpy as np
import pickle
import warnings
import matplotlib.pyplot as plt
import datetime
import os
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from datetime import date
import pathlib

from config.definitions import ROOT_DIR

# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html
#from sklearn.metrics import balanced_accuracy_score

from pandas import DataFrame
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
             
logging.info("Python libraries loaded")

In [ ]:
# create folders
process_path1 = os.path.join('data','processed',"")

isExist = os.path.exists(process_path1)
if isExist == False:
    create_path = pathlib.Path(process_path1)
    create_path.mkdir(parents=True, exist_ok=True)
    
    
# create folders
process_path2 = os.path.join('data','output',"")

isExist = os.path.exists(process_path2)
if isExist == False:
    create_path = pathlib.Path(process_path2)
    create_path.mkdir(parents=True, exist_ok=True)

**IMPORTANT**

Load actual vs predicted results into the processed folder before continuing
+ Column Names:text, actual_intent, pred_intent, pred_conf
+ File Type: Excel



In [ ]:
#Intialize Values

# path to load current and previous files
# defaults to getting the last two recent files. If you want to change the files can manually enter them here.

k_fold_files = os.listdir(process_path1)
k_fold_files.sort()

current_file_path = process_path1
current_file_name = 'pwc_2022_06_29.xlsx'
#current_file_name = k_fold_files[-1]

prev_file_path = process_path1
prev_file_name = 'pwc_2022_06_29.xlsx'
#prev_file_name = k_fold_files[-2]

# intents to ignore for the report
intents_to_ignore = ['Default Fallback Intent', 'Default Welcome Intent']

# threshold values for flags
support_less_than = 30
f1_less_than = .67
wrong_prediction_percent_greater_than = .30
class_imbalance_ratio_greater_than = 20

logging.info("Parameters initialized")

In [ ]:
# function to build tables for analysis

def create_analysis_tables(dataframe_input_list):

    '''
    Overivew: A function to build tables that will be used for the analysis
    Depends On Function: None
    Constraints: You must have panda dataframes in a list. Each panda dataframe should be formatted:
        text (str), acutal_intent(str), pred_intent (str), pred_conf (float), test_num (int)
    Input:
        1. A list of panda dataframes (list)
    Output:
        1. df_intents (dataframe):
        2. df_conf_mat (dataframe):
        3. df_summary (dataframe):
        4. df_plot (dataframe):
        5: df_plot_wrong_grouped (dataframe): 
    '''
    
    if type(dataframe_input_list) == list:
        all_data  = pd.concat(dataframe_input_list)
        df_intents = all_data[['text','actual_intent','pred_intent', 'pred_conf']]
    else:
        all_data = dataframe_input_list
        df_intents = dataframe_input_list
    
    
    '''
    df_intents is table that is tidy data of the test data results
    Ex: This phrase, was supposed to go to this intent, but went to this this intent instead with this confident score from dialogflow.
    '''
    
    
    '''
    df_conf_mat is table that gives the number of times a predicted intent occured for an actual intent
    Ex: The predicted intent cme.3.1-callme_cancel got predicted five times for the actual intent !cancel_intent
    '''
    df_conf_mat = (df_intents
      .groupby(["actual_intent", "pred_intent"])
      .agg(n_pred=("pred_conf", "size"))
      .reset_index())
    
    
    '''
    df_summary is a table that gives the number of predictions made for an intent
    Ex: !cancel_intent got predicted 50 times in the test data
    '''
    df_summary = (df_intents
     .groupby('pred_intent')
     .agg(n=('pred_conf', 'size'),
          mean_conf=('pred_conf', 'mean')))
    df_summary = df_summary.rename(columns={'n':'predictions', 'mean_conf':'df_conf'})
    
    
    '''
    df_plot is a table that combines the tables of df_conf_mat and df_summary into one table.
    So you can see how many times dilogflow predicted an intent for certain intent and the overall number of times that intent was predicted.
    Ex: Dialogflow predicted cme.3.1-callme_cancel five times for the actual intent !cancel_intent. Overall cme.3.1-callme_cancel was
        predicted forty-three times by Dialogflow. Of those forty-three predictions, P means it predicted cme.3.1-callme_cancel 11.6% of those
        43 predictions for the intent !cancel_intent. 

    This helps to understand the distribution of the predictions and where model predicted incorrectly or correctly often.
    '''
    df_plot = df_conf_mat.merge(df_summary.reset_index()).assign(p=lambda d: d['n_pred']/d['predictions'])
    
    
    '''
    df_plot_wrong_grouped is a table that counts how many times dialogflow made a prediction, that prediction was incorrect.
    Ex: !cancel_intent was predicted 50, of those 50 times it was predicted, it was predicted wrong 50 times.
    '''
    df_plot['correct_flag'] = np.where(df_plot['actual_intent'] == df_plot['pred_intent'], 1,0)
    df_plot_wrong = df_plot[df_plot['correct_flag'] == 0]
    df_plot_wrong_grouped = df_plot_wrong.groupby('pred_intent')['n_pred'].sum().reset_index().set_index('pred_intent')
    df_plot_wrong_grouped = df_plot_wrong_grouped.rename(columns={'n_pred':'FP'})
     
    return all_data, df_intents, df_conf_mat, df_summary, df_plot, df_plot_wrong_grouped
    #incorrectly_predicted = FP

def compute_overall_f1_score(dataframe, ignore_intents=[]):
    '''
    **THIS IS CURRENTLY NOT USED, BUT HERE FUTURE USE IN CASE EVER NEEDED**
    The is a fuction to computer the overall F1-score and class balance score using the macro, 
    micro and weighted to create an average score. 
    '''
    
    dataframe = dataframe[~dataframe.index.isin(ignore_intents)]
    test_y = dataframe['actual_intent'].values
    preds = dataframe['pred_intent'].values
    
    macro_f1 = f1_score(test_y, preds, average='macro', labels=np.unique(preds))
    micro_f1 = f1_score(test_y, preds, average='micro', labels=np.unique(preds))
    weighted_f1 = f1_score(test_y, preds, average='weighted', labels=np.unique(preds))

    combined_avg = (macro_f1+micro_f1+weighted_f1)/3
    combined_avg = round(combined_avg,4) * 100
    
    balance_score = balanced_accuracy_score(test_y, preds, adjusted=True)
    return combined_avg, balance_score


def built_report(all_dataframe, summary_dataframe, wrong_group_dataframe, confusion_matrix, ignore_intents=[]):
    '''
    This is a function to combine dataframes for each k-fold data
    '''
    test_y = all_dataframe['actual_intent'].values
    preds = all_dataframe['pred_intent'].values
    report_dict = classification_report(test_y, preds, output_dict=True, labels=np.unique(preds))
    result_df = pd.DataFrame(report_dict).transpose()
    
    # removes the last three summary f1 scores. Uncomment out to add back, but will mess up downstream reports
    result_df = result_df.iloc[:-3]
    
    result_df = result_df.merge(summary_dataframe, how='left', left_index=True, right_index=True)
    
    #add false positive and false negative counts to intents
    fp_df = confusion_matrix['Actual'].transpose()
    fp_s = fp_df['FP Total']

    fn_df = confusion_matrix['Actual']
    fn_s = fn_df['FN Total']
    
    result_df = result_df.merge(fp_s, how='left', left_index=True, right_index=True)
    result_df = result_df.merge(fn_s, how='left', left_index=True, right_index=True)
    result_df = result_df.rename(columns={'FP Total':'fp', 'FN Total':'fn'})
    
    #ignore intents
    result_df = result_df[~result_df.index.isin(ignore_intents)]
    
    # add calculations
    result_df['fp_%'] = (result_df['fp'] / result_df['predictions'])
    result_df['fn_%'] = (result_df['fn'] / result_df['predictions'])
    result_df['prec_recall_delta'] = abs(result_df['precision'] - result_df['recall'])
    result_df['f1_df_predict_delta'] = abs(result_df['f1-score'] - result_df['df_conf'])
    result_df['support_ratio'] = result_df['support'].max() / result_df['support']
    return result_df

def build_history_report(current_dataframe, previous_dataframe):
    '''
    This is function that combines the build_report function to help compare the previous k-fold report to the current 
    k-fold report.
    '''
    # merge the two reports
    history_df = current_report.merge(previous_report, how='left', left_index=True, right_index=True, suffixes=('_cur', '_prev'))
    
    # compute differences
    history_df['precision_diff'] = history_df['precision_cur'] - history_df['precision_prev']
    history_df['recall_diff'] = history_df['recall_cur'] - history_df['recall_prev']
    history_df['f1-score_diff'] = history_df['f1-score_cur'] - history_df['f1-score_prev']
    history_df['df_conf_diff'] = history_df['df_conf_cur'] - history_df['df_conf_prev']
    history_df['fp_diff'] = history_df['fn_cur'] - history_df['fn_prev']
    history_df['fp_%_diff'] = history_df['fn_%_cur'] - history_df['fn_%_prev']
    history_df['fn_diff'] = history_df['fn_cur'] - history_df['fn_prev']
    history_df['fn_%_diff'] = history_df['fn_%_cur'] - history_df['fn_%_prev']
    
    # rename columns
    rename_dict = {'support_cur':'support',
                   'support_ratio_cur':'support_ratio'
                  }
    history_df = history_df.rename(columns=rename_dict)
    
    #organize the columns
    cols = ['support','support_ratio','predictions_cur','fp_cur','fp_prev','fp_diff','fn_cur','fn_prev','fn_diff','precision_cur','precision_prev','precision_diff','recall_cur','recall_prev','recall_diff','f1-score_cur','f1-score_prev','f1-score_diff',
        'df_conf_cur','df_conf_prev','df_conf_diff','fp_%_cur','fp_%_prev','fp_%_diff', 'fn_%_cur','fn_%_prev','fn_%_diff']
    
    history_df = history_df[cols]
    return history_df

def add_flags(dataframe, support_flag_value, f1_flag_value, wrong_pred_flag_value, imbalance_ratio_flag):
    '''
    This is a function to add flags to the history dataframe 
    **NOTE**: can only be run once otherwise duplicates flag total values
    '''
    support_less_than = support_flag_value
    f1_less_than = f1_flag_value
    wrong_prediction_percent_greater_than = wrong_pred_flag_value
    
    # intents that need more training examples low support intents
    dataframe['support_flag'] = np.where(dataframe['support'] < support_less_than, 1,0)

    # intents with support greater than 30 training samples, but f1 lower than threshold
    dataframe['f1_flag'] = np.where( (dataframe['support'] >= support_less_than) & (dataframe['f1-score_cur'] < f1_less_than), 1,0)

    # intents with support greater than 30 training samples and difference between f1-score and dialogflow confidence meets threshold
    #dataframe['wrong_predict_flag'] = np.where( (dataframe['support'] >= support_less_than) & (dataframe['incorrectly_predicted_cur'] > wrong_prediction_percent_greater_than), 1,0)
    
    # class imbalance flag
    dataframe['class_imbalance_flag'] = np.where( dataframe['support_ratio'] >= imbalance_ratio_flag, 1,0)

    # intents with support greater than 30 training samples and difference between f1-score and dialogflow confidence meets threshold
    dataframe['flag_total'] = dataframe.iloc[:, -3:].sum(axis=1)

    cols_to_add = ['support_flag', 'f1_flag','flag_total']
    flag_totals = dataframe[cols_to_add].transpose().iloc[:, :].sum(axis=1).reset_index().rename(columns={'index':'flag',0:'total'})
    return dataframe, flag_totals


def build_prediction_matrix(k_fold_data,  ignore_intents=[]):
    '''
    Overview: A function to create a confusion matrix from k-fold cross-validation data
    Input: 
        1. k_fold_data (dataframe): The Dialogflow k-fold test data
    Output:
        1. prediction_maxtrix_df (dataframe): The confusion matrix
    '''
    
    y_true = k_fold_data['actual_intent'].values
    y_pred = k_fold_data['pred_intent'].values
    labels = [x for x in np.unique(y_pred)]
    cf_m = confusion_matrix(y_true, y_pred, labels=labels)
    df_cm = DataFrame(cf_m, index=labels, columns=labels)
    df_cm = df_cm[~df_cm.index.isin(ignore_intents)]
    prediction_maxtrix_df = df_cm.reset_index()
    return prediction_maxtrix_df
    
    
def build_total_matrix(confusion_matrix_df):
    '''
    Overview: A function to output a confusion matrix with totals and multi-index columns for output
    Input:
        1. confusion_matrix_df (dataframe): A standard confusion matrix created from sklearn.
    Output:
        1. matrix_totals (dataframe)
    '''
    matrix_totals = confusion_matrix_df
    matrix_totals = matrix_totals.rename(columns={'index':'Expected'})
    matrix_totals.loc[:,'Expected Total'] = matrix_totals.sum(axis=1)
    header = pd.MultiIndex.from_product([['Actual'],list(matrix_totals.columns)])
    matrix_totals = pd.DataFrame(data=matrix_totals.iloc[:,1:].to_numpy(), index=matrix_totals['Expected'], columns = header[1:])
    matrix_totals.loc['Actual Total',:]= matrix_totals.sum(axis=0)
    return matrix_totals

def build_true_false_matrix(confusion_matrix_df):
    '''
    Overview: A function to output a confusion matrix with totals and multi-index columns for output
    Input:
        1. confusion_matrix_df (dataframe): A standard confusion matrix created from sklearn.
    Output:
        1. matrix_totals (dataframe)
    '''
    matrix_totals = confusion_matrix_df
    matrix_totals = matrix_totals.set_index('index')
    np.fill_diagonal(matrix_totals.values, 0)
    matrix_totals = matrix_totals.reset_index(drop=False)
    matrix_totals = matrix_totals.rename(columns={'index':'Expected'})
    matrix_totals.loc[:,'FN Total'] = matrix_totals.sum(axis=1)
    header = pd.MultiIndex.from_product([['Actual'],list(matrix_totals.columns)])
    matrix_totals = pd.DataFrame(data=matrix_totals.iloc[:,1:].to_numpy(), index=matrix_totals['Expected'], columns = header[1:])
    matrix_totals.loc['FP Total',:]= matrix_totals.sum(axis=0)
    return matrix_totals


logging.info("Functions created")   

In [ ]:
# load files
#current_df = pd.read_pickle(current_file_path+current_file_name)
#prev_df = pd.read_pickle(prev_file_path+prev_file_name)

current_df = pd.read_excel(current_file_path+current_file_name)
prev_df = pd.read_excel(prev_file_path+prev_file_name)

logging.info("Current and previous files loaded")
#current_df[0].head()

In [ ]:
# uses the functions in create_analysis_tables to build dataframes for the report

cur_all_df, cur_intents_df, cur_conf_mat_df, cur_summ_df, cur_plot_df, cur_plot_wrong_grouped_df = create_analysis_tables(current_df)
prev_all_df,prev_intents_df, prev_conf_mat_df, prev_summ_df, prev_plot_df, prev_plot_wrong_grouped_df = create_analysis_tables(prev_df)

logging.info("Current and previous tables for analysis created") 

In [ ]:
# build confusion matrices for current data

cur_prediction_matrix = build_prediction_matrix(cur_all_df)
cur_prediction_matrix_total = build_total_matrix(cur_prediction_matrix)
cur_prediction_matrix_zeroed = build_true_false_matrix(cur_prediction_matrix)
#cur_prediction_matrix_zeroed

In [ ]:
# build confusion matrices for previous data

prev_prediction_matrix = build_prediction_matrix(prev_all_df)
prev_prediction_matrix_total = build_total_matrix(prev_prediction_matrix)
prev_prediction_matrix_zeroed = build_true_false_matrix(prev_prediction_matrix)
#prev_prediction_matrix_zeroed

In [ ]:
# track deleted and added phrases
prev_all_df['index'] = prev_all_df['text']+ prev_all_df['actual_intent']
cur_all_df['index'] = cur_all_df['text']+ cur_all_df['actual_intent']

prev_all_df = prev_all_df.set_index('index')
cur_all_df = cur_all_df.set_index('index')
merged = prev_all_df.merge(cur_all_df, how='outer', left_index=True, right_index=True, suffixes=('_prev', '_cur'), indicator=True)

#phrases added
cols = ['text_cur','actual_intent_cur']
phrases_added = merged[cols][merged['_merge']=='right_only'].reset_index(drop=True)
phrases_added = phrases_added.rename(columns={'text_cur':'deleted text', 'actual_intent_cur':'intent'})

#phrases deleted or changed
cols = ['text_prev', 'actual_intent_prev'] 
phrases_deleted_or_changed = merged[cols][merged['_merge']=='left_only'].reset_index(drop=True)
phrases_deleted_or_changed = phrases_deleted_or_changed.rename(columns={'text_prev':'added text', 'actual_intent_prev':'intent'})


In [ ]:
# builds the report for the previous and current data

ignore_intents = []
#ignore_intents = intents_to_ignore
#ignore_knowledge_intents_list = list(set(cur_all_df[cur_all_df['pred_intent'].str.startswith('Knowledge.')==True]['pred_intent'])) #ignoring the knowledge intents
#ignore_intents = ignore_knowledge_intents_list + ignore_intents

current_report = built_report(cur_all_df, cur_summ_df, cur_plot_wrong_grouped_df, cur_prediction_matrix_zeroed, ignore_intents=ignore_intents)
previous_report = built_report(prev_all_df, prev_summ_df, prev_plot_wrong_grouped_df, prev_prediction_matrix_zeroed, ignore_intents=ignore_intents)

logging.info("Current and previous reports created") 

In [ ]:
# combines the previous and current data into one dataframe for comparison

history_df = build_history_report(current_report, previous_report)

logging.info("History table created")
#history_df

In [ ]:
# buidling the summary table that will be outputed to compare the previous and current data

current_f1 = history_df['f1-score_cur'].mean()
previous_f1 = history_df['f1-score_prev'].mean()
f1_difference = current_f1 - previous_f1

current_total_fp = history_df['fp_cur'].sum()
previous_total_fp = history_df['fp_prev'].sum()
fp_difference = current_total_fp - previous_total_fp

current_total_fn = history_df['fn_cur'].sum()
previous_total_fn = history_df['fn_prev'].sum()
fn_difference = current_total_fn - previous_total_fn


current_balance = history_df['support_ratio'].mean()
phrases_added_sum = phrases_added.shape[0]
phrased_chngd_or_del_sum = phrases_deleted_or_changed.shape[0]

summary_values = [current_f1, previous_f1, f1_difference, current_total_fp, previous_total_fp, fp_difference,
                  current_total_fn, previous_total_fn, fn_difference, current_balance, phrases_added_sum, phrased_chngd_or_del_sum]

index_vals = ['Current F1 Avg', 'Previous F1 Avg', 'F1 Diff',
              'Current FP Total', 'Previous FP Total', 'FP Diff', 
              'Current FN Total', 'Previous FN Total', 'FN Diff',
              'Current Class Imbalance Avg',
             'Phrases Added or Changed', 'Phrases Removed or Changed']

today = date.today()
columns_vals = [today]

f1_summary_df = pd.DataFrame(data=summary_values, columns=columns_vals, index=index_vals)

logging.info("Summary table created") 
#f1_summary_df

In [ ]:
# building the descriptive stats table for output

history_stats = history_df.describe()

logging.info("Descriptive stats of history table created") 
#history_stats

In [ ]:
#add the flags to the history dataframe

flags_df, flag_total_df = add_flags(history_df, support_less_than, 
                                    f1_less_than, 
                                    wrong_prediction_percent_greater_than,
                                   class_imbalance_ratio_greater_than)

flags_df = flags_df.sort_values(by=['f1-score_cur','flag_total', ], ascending=[True,False])
logging.info("Flags added to history table") 
#flags_df

In [ ]:
# outputing all the files to Excel

now = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
file_name = 'cv_scan'+now+'.xlsx'


file_path = os.path.join('data','output','cross_validation_intent_level',"")

#file_path = 'data/output/cross_validation_intent_level/'

with pd.ExcelWriter(file_path+file_name) as writer: 
    f1_summary_df.to_excel(writer, sheet_name='overall_summary', index=True)
    history_stats.to_excel(writer, sheet_name='descriptive_summary', index=True)
    flags_df.to_excel(writer, sheet_name='intent_summary', index=True)
    cur_prediction_matrix_total.to_excel(writer, sheet_name='prediction_matrix_totals', index=True)
    cur_prediction_matrix_zeroed.to_excel(writer, sheet_name='FP_FN_matrix', index=True)
    phrases_added.to_excel(writer, sheet_name='added_changed_phrases', index=False)
    phrases_deleted_or_changed.to_excel(writer, sheet_name='deleted_changed_phrases', index=False)
    cur_all_df.to_excel(writer, sheet_name='current_all_data', index=False)
    prev_all_df.to_excel(writer, sheet_name='previous_all_data', index=False)
    cur_prediction_matrix.to_excel(writer, sheet_name='prediction_matrix', index=False)
    
logging.info("Excel output file created") 

In [ ]:
logging.info("building reports for k-fold cross-validation complete")

In [ ]:
cur_prediction_matrix